In [1]:
import numpy as np
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,MaxPooling2D
from keras.layers import Dense,Flatten,SpatialDropout2D
from keras.layers.merge import concatenate

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import History
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

Using TensorFlow backend.


In [8]:
def buildNet(num_classes=2):
    """
    Function to build 4 layer NN with 2 Conv layers, 1 MaxPool layer,
    1 GlobalMaxPool layer and 2 Dense layers
    
    Parameters
    ----------
    num_classes: int
                 Number of classes in training data
    Returns
    -------
    model:
            Model created
           
    """
    #channel1 RGB Data
    input1= Input(shape=(64,64,3))
    conv11=Convolution2D(32, (3,3),input_shape=(64,64,3),activation='relu')(input1)
    pool11=MaxPooling2D(pool_size=(2,2))(conv11)
    conv12=Convolution2D(64, (3,3),activation='relu')(pool11)
    pool12=GlobalAveragePooling2D()(conv12)
    
    #Channel2 Greyscale
    input2= Input(shape=(64,64,1))
    conv21=Convolution2D(32, (3,3),input_shape=(64,64,3),activation='relu')(input2)
    pool21=MaxPooling2D(pool_size=(2,2))(conv21)
    conv22=Convolution2D(64, (3,3),activation='relu')(pool21)
    pool22=GlobalAveragePooling2D()(conv22)
    
    merge=concatenate([pool12,pool22])
    hidden1=Dense(128, activation='relu')(merge)
    output=Dense(num_classes, activation='sigmoid')(hidden1)
    model=Model(input=[input1,input2],outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [10]:
def plot_model_history(history):
    """
    Function to plot training and validation data of model
    
    Parameters
    ----------
    history: dictionary
             history of training and validation of model
    Returns
    -------
    None
    
    """
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def trainNet(X_train1,X_train2,y_train,X_val1,X_val2,y_val):
    """
    Function to train NN created
    
    Arguments
    ---------
    X_train: numpy array
             training samples
    y_train: numpy array
             labels of training samples
    
    X_val: numpy array
             validation samples
    y_val: numpy array
             labels of validation samples
    
    Returns
    -------
    None
    """
    num_classes = y_train.shape[1]
    model = buildNet(num_classes)
    history = History()
    callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),history]
    model.fit([X_train1,X_train2], y_train, validation_data=([X_val1,X_val2], y_val), epochs=30, batch_size=100,callbacks=callbacks,verbose=1)
    model.save("MultiChannel_Retinopathy.h5")
    plot_model_history(history)

In [11]:
def testNet(X_test,y_test):
    """
    Function to load and test the model and print confusion matrix
    
    Parameters
    ----------
    X_test: numpy array
            testing samples
    y_test: numpy array
            labels of testing samples
    Returns
    -------
    pred: list
          prediction made by model
    """
    model = load_model('best_model.h5')
    pred=model.predict([X_test1,X_test2])
    pred=np.argmax(np.round(pred),axis=1)
    actual=np.argmax(np.round(y_test),axis=1)
    conf_matrix=confusion_matrix(actual, pred)
    print(conf_matrix)

In [ ]:
if __name__=="__main__":
    """
    1.Add code to shuffle data and preprocess image data
    2.Split data into train,test sets
    3.Uncomment the code below 
    """
    #hist=trainNet(X_train,y_train,X_val,y_val)
    #plot_model_history(hist)
    #testNet(X_val,y_val)
    